In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib
import h5py

In [ ]:
Ls = np.array(
    [
        [4, 16, 16, 16],
        [4, 20, 20, 20],
        [4, 24, 24, 24],
        [4, 28, 28, 28],
        [4, 32, 32, 32],
    ]
)

# Set plotting options
plt.rcParams["font.size"] = "10"
plt.rcParams["figure.figsize"] = (5, 5)
cmap = matplotlib.colormaps["viridis"]
cm_subsection = np.array(np.linspace(0, 200, len(Ls)), dtype=int)
colors = [cmap.colors[x] for x in cm_subsection]
fmt = ["_", "*", "p", "D", "^", "s", "o"]
markersize = 12
capsize = 3

# Number of lattice sites = volume
vol_tot = np.prod(Ls, axis=1)

# spatial volume
vol = np.prod(Ls[:,1:], axis=1)

# Number of Samples
N = 600

bootstrap_seed = 9999

# Reweighting parameters: Set number of bootstraps, reweighting convergence tolerance, reweighting partition and dati header
N_bootstraps = 2000
tol = 1e-14
n_part = 50
dati_head = 5 

# Literature critical beta 5.69236(15)
literature_beta = 5.69236
lit_beta_min = 5.69221
lit_beta_max = 5.69251

In [ ]:
os.makedirs(f"../reports/Nt={Ls[0,0]}/figures/", exist_ok=True)

# Reweight observables

In [ ]:
###Uncomment following two lines for linux compilation
# %%bash
# python3 setup.py build_ext --inplace

###Uncomment following two lines for cmd compilation
# !python setup.py build_ext --inplace -v

In [ ]:
import reweight
print(reweight.__file__)

In [ ]:
os.makedirs(f"../cached_data/", exist_ok=True)

Reweight $\rho_0$

In [ ]:
# Reweighting beta values for each lattice size
## to stabilise numerics for smaller lattices sizes
betas_k = [
    np.array(
        [
            5.68, 5.6825, 5.685, 5.6875, 5.69, 5.6925, 5.695, 5.6975, 5.7, 5.705
        ]
    ),
    np.array(
        [
            5.675, 5.6800, 5.685, 5.6900, 5.691, 5.692, 5.693, 5.694, 5.695, 5.7000
        ]
    ),
    np.array(
        [
            5.691, 5.6913, 5.6915, 5.6917, 5.6919, 5.6921, 5.6923, 5.6925, 5.693, 5.6935
        ]
    ),
    np.array(
        [
            5.691, 5.6915, 5.6918, 5.692, 5.6921, 5.6922, 5.6923, 5.6925, 5.6928, 5.693
        ]
    ),
    np.array(
        [
            5.6915, 5.692, 5.6922, 5.6924, 5.6926, 5.6927, 5.6928, 5.693, 5.6932, 5.6935
        ]
    ),
]


In [ ]:
# Error bars are computed via the bootstrapping method

# Generate bootstraps
np.random.seed(bootstrap_seed)
bootstraps = [
    [
        [
            np.sort(np.random.choice(range(N), size=N, replace=True))
            for i in range(N_bootstraps)
        ]
        for b in range(len(betas_k[l]))
    ]
    for l in range(len(Ls))
]

In [ ]:
h = []
for l in range(len(Ls)):
    h_l = []
    for b in betas_k[l]:
        # Load Betti number HDF5 file if exists (else compute and save)
        filename = f"../data/observables/betti/{Ls[l][0]}.{Ls[l][1]}.{Ls[l][2]}.{Ls[l][3]}/betti_NtNsNsNs={Ls[l][0]}{Ls[l][1]}{Ls[l][2]}{Ls[l][3]}_b={b:.4f}.csv"
        # print(filename)
        if os.path.exists(filename):
            h_b = np.loadtxt(filename, delimiter=",", dtype=int)
        h_l.append(h_b)
    h.append(h_l)

In [ ]:
# import action data
# Skip header line + start at sweep 10_000
E_l = []
for l in range(len(Ls)):
    print(Ls[l])
    E_b = []
    for b in range(len(betas_k[l])):
        # print(betas_k[l][b])
        averages = []
        with open(
            f"../data/observables/dati/{Ls[l][0]}.{Ls[l][1]}.{Ls[l][2]}.{Ls[l][3]}/dati.dat{betas_k[l][b]:.4f}",
            "r",
        ) as file:
            for line in file:
                elements = line.split()
                averages.append(
                    1.0 - (0.5 * (np.float64(elements[0]) + np.float64(elements[1])))
                )
        total_action = 6.0 * vol_tot[l] * np.array(averages, dtype=np.float64)
        E_b.append(total_action[dati_head : (dati_head + N)])
    E_l.append(E_b)
E = E_l

In [ ]:
# Compute delta_S for use in multiple histogram reweighting
E_bar = [np.mean(E[l], axis=0) for l in range(len(Ls))]
delta_E = [E[l] - E_bar[l][np.newaxis, :] for l in range(len(Ls))]

In [ ]:
# Generate b1 bootstrap distribution
b0_bootstrap = [
    [
        np.array([h[l][b][bootstraps[l][b][i]][:, 0] for b in range(len(betas_k[l]))])
        for l in range(len(Ls))
    ]
    for i in range(N_bootstraps)
]

In [ ]:
# whole range of betas
rw_betas_linear = [
    np.linspace(betas_k[l][0], betas_k[l][-1], n_part, dtype=np.float64) for l in range(len(Ls))]

In [ ]:
rw_means_linear = []
rw_vars_linear = []
rw_means_linear_err = []
rw_vars_linear_err = []
pseudo_bcs_b0_bs = []
for l in range(len(Ls)):
    print(l)
    filename = f"../cached_data/cache_rw_{np.min(rw_betas_linear[l]):.4f}-{np.max(rw_betas_linear[l]):.4f}_NtNsNsNs={Ls[l][0]}{Ls[l][1]}{Ls[l][2]}{Ls[l][3]}_b0.h5"
    # print(filename)
    if os.path.exists(filename):
        with h5py.File(filename, "r") as hf:
            rw_means_linear.append(hf["rw_means_linear"][()])
            rw_means_linear_err.append(hf["rw_means_linear_err"][()])
            rw_vars_linear.append(hf["rw_vars_linear"][()])
            rw_vars_linear_err.append(hf["rw_vars_linear_err"][()])
            pseudo_bcs_b0_bs.append(hf["pseudo_bc_bs"][()])
    else:
        temp = np.array(h[l])
        rw_means_linear_temp, rw_means_linear_temp_err, rw_vars_linear_temp, rw_vars_linear_temp_err, pseudo_bc_temp_bs = reweight.reweight(
            temp[:, :, 0], delta_E[l], betas_k[l], rw_betas_linear[l], N_bootstraps=N_bootstraps, tol=tol
        )
        with h5py.File(filename, "w") as hf:
            hf.create_dataset("rw_means_linear", data=rw_means_linear_temp)
            hf.create_dataset("rw_means_linear_err", data=rw_means_linear_temp_err)
            hf.create_dataset("rw_vars_linear", data=rw_vars_linear_temp)
            hf.create_dataset("rw_vars_linear_err", data=rw_vars_linear_temp_err)
            hf.create_dataset("pseudo_bc_bs", data=pseudo_bc_temp_bs)
        rw_means_linear.append(rw_means_linear_temp)
        rw_vars_linear.append(rw_vars_linear_temp)
        rw_means_linear_err.append(rw_means_linear_temp_err)
        rw_vars_linear_err.append(rw_vars_linear_temp_err)
        pseudo_bcs_b0_bs.append(pseudo_bc_temp_bs)
rw_means_linear_b0 = np.array(rw_means_linear)
rw_vars_linear_b0 = np.array(rw_vars_linear)
rw_means_linear_err_b0 = np.array(rw_means_linear_err)
rw_vars_linear_err_b0 = np.array(rw_vars_linear_err)
pseudo_bcs_b0_bs = np.array(pseudo_bcs_b0_bs)
rw_means_linear_b0.shape, rw_means_linear_err_b0.shape, rw_vars_linear_b0.shape, rw_vars_linear_err_b0.shape, pseudo_bcs_b0_bs.shape

In [ ]:
for l in range(len(Ls)):
    plt.scatter(rw_betas_linear[l], rw_means_linear_b0[l] / vol[l], s=1, color=colors[l], alpha=0.1)
    plt.errorbar(rw_betas_linear[l], rw_means_linear_b0[l] / vol[l], ls="None", yerr=rw_means_linear_err_b0[l] / vol[l], capsize=0, color=colors[l], alpha=0.05)
    x = betas_k[l]
    temp = np.array(h[l])
    y = np.mean(temp[:, :, 0], axis=1) / vol[l]
    plt.scatter(
        x,
        y,
        s=markersize,
        marker=fmt[l],
        label=r"$Nt, Ns = " + str(int(Ls[l][0])) + "," + str(int(Ls[l][1])) + "$",
        color=colors[l],
    )
    means = [
        np.mean(b0_bootstrap[i][l] , axis=1) / vol[l] for i in range(len(b0_bootstrap))
    ]
    yerr = np.std(means, axis=0)
    plt.errorbar(x, y, ls="None", yerr=yerr, capsize=2, color=colors[l], alpha=0.25)

plt.xlabel(r"$\beta$")
plt.ylabel(r"$\rho_{0}$")
plt.legend(loc="best")
plt.tight_layout()
plt.locator_params(axis="x", nbins=10)
ax = plt.gca()

# plt.savefig("../reports/figures/p_0_scatter_rw.pdf", format="pdf")
plt.show()

In [ ]:
for l in range(len(Ls)):
    plt.scatter(rw_betas_linear[l], (rw_vars_linear_b0[l]/vol[l]), s=1, color=colors[l], alpha=0.1)
    plt.errorbar(rw_betas_linear[l], (rw_vars_linear_b0[l]/vol[l]), ls="None", yerr=rw_means_linear_err_b0[l]/np.sqrt(vol[l]), capsize=0, color=colors[l], alpha=0.05)
    x = betas_k[l]
    temp = np.array(h[l])
    y = np.var(temp[:, :, 0], axis=1) / vol[l]
    plt.scatter(
        x,
        y,
        s=markersize,
        marker=fmt[l],
        label=r"$Nt, Ns = " + str(int(Ls[l][0])) + "," + str(int(Ls[l][1])) + "$",
        color=colors[l],
    )
    vars = [
        np.var(b0_bootstrap[i][l], axis=1) / vol[l]  for i in range(len(b0_bootstrap))
    ]
    yerr = np.std(vars, axis=0)
    plt.errorbar(x, y, ls="None", yerr=yerr, capsize=capsize, color=colors[l], alpha=0.25)

plt.xlabel(r"$\beta$")
plt.ylabel(r"$\chi_{0}$")
plt.legend(loc="best")
plt.tight_layout()
plt.locator_params(axis="x", nbins=10)
ax = plt.gca()
# plt.savefig("../reports/figures/X_0_scatter_rw.pdf", format="pdf")
plt.show()

In [ ]:
yerr_b0 = np.std(pseudo_bcs_b0_bs,axis=1)
pseudo_bcs_b0 = np.mean(pseudo_bcs_b0_bs,axis=1)
yerr_b0, pseudo_bcs_b0

In [ ]:
x = 1/vol
y = pseudo_bcs_b0
plt.scatter(x, y, s=10)
plt.errorbar(x, y, yerr=yerr_b0, fmt='.',capsize=3)
plt.axhspan(lit_beta_min, lit_beta_max, color="purple", alpha=0.15)
plt.axhline(literature_beta, color="purple", alpha=0.3)

Reweight $\rho_1$

In [ ]:
# Reweighting beta values for each lattice size
## to stabilise numerics for smaller lattices sizes
betas_k = [
    np.array(
        [
            5.68, 5.6825, 5.685, 5.6875, 5.69, 5.6925, 5.695, 5.6975, 5.7, 5.705
            
        ]
    ),
    np.array(
        [
            5.675, 5.68, 5.685, 5.69, 5.691, 5.692, 5.693, 5.694, 5.695, 5.7
        ]
    ),
    np.array(
        [
            5.691, 5.6913, 5.6915, 5.6917, 5.6919, 5.6921, 5.6923, 5.6925, 5.693, 5.6935
        ]
    ),
    np.array(
        [
            5.691, 5.6915, 5.6918, 5.692, 5.6921, 5.6922, 5.6923, 5.6925, 5.6928, 5.693
        ]
    ),
    np.array(
        [
            5.6915, 5.692, 5.6922, 5.6924, 5.6926, 5.6927, 5.6928, 5.693, 5.6932, 5.6935
        ]
    ),
]


In [ ]:
# Error bars are computed via the bootstrapping method

# Generate bootstraps
np.random.seed(bootstrap_seed)
bootstraps = [
    [
        [
            np.sort(np.random.choice(range(N), size=N, replace=True))
            for i in range(N_bootstraps)
        ]
        for b in range(len(betas_k[l]))
    ]
    for l in range(len(Ls))
]

In [ ]:
h = []
for l in range(len(Ls)):
    h_l = []
    for b in betas_k[l]:
        # Load Betti number HDF5 file if exists (else compute and save)
        filename = f"../data/observables/betti/{Ls[l][0]}.{Ls[l][1]}.{Ls[l][2]}.{Ls[l][3]}/betti_NtNsNsNs={Ls[l][0]}{Ls[l][1]}{Ls[l][2]}{Ls[l][3]}_b={b:.4f}.csv"
        # print(filename)
        if os.path.exists(filename):
            h_b = np.loadtxt(filename, delimiter=",", dtype=int)
        h_l.append(h_b)
    h.append(h_l)

In [ ]:
E_l = []
for l in range(len(Ls)):
    print(Ls[l])
    E_b = []
    for b in range(len(betas_k[l])):
        # print(betas_k[l][b])
        averages = []
        with open(
            f"../data/observables/dati/{Ls[l][0]}.{Ls[l][1]}.{Ls[l][2]}.{Ls[l][3]}/dati.dat{betas_k[l][b]:.4f}",
            "r",
        ) as file:
            for line in file:
                elements = line.split()
                averages.append(
                    1.0 - (0.5 * (np.float64(elements[0]) + np.float64(elements[1])))
                )
        total_action = 6.0 * vol_tot[l] * np.array(averages, dtype=np.float64)
        E_b.append(total_action[dati_head : (dati_head + N)])
    E_l.append(E_b)
E = E_l

In [ ]:
# Compute delta_S for use in multiple histogram reweighting
E_bar = [np.mean(E[l], axis=0) for l in range(len(Ls))]
delta_E = [E[l] - E_bar[l][np.newaxis, :] for l in range(len(Ls))]

In [ ]:
# Generate b1 bootstrap distribution
b1_bootstrap = [
    [
        np.array([h[l][b][bootstraps[l][b][i]][:, 1] for b in range(len(betas_k[l]))])
        for l in range(len(Ls))
    ]
    for i in range(N_bootstraps)
]

In [ ]:
# whole range of betas
rw_betas_linear = [
    np.linspace(betas_k[l][0], betas_k[l][-1], n_part, dtype=np.float64) for l in range(len(Ls))]

In [ ]:
rw_means_linear = []
rw_vars_linear = []
rw_means_linear_err = []
rw_vars_linear_err = []
pseudo_bcs_b1_bs = []
for l in range(len(Ls)):
    print(l)
    filename = f"../cached_data/cache_rw_{np.min(rw_betas_linear[l]):.4f}-{np.max(rw_betas_linear[l]):.4f}_NtNsNsNs={Ls[l][0]}{Ls[l][1]}{Ls[l][2]}{Ls[l][3]}_b1.h5"
    if os.path.exists(filename):
        with h5py.File(filename, "r") as hf:
            rw_means_linear.append(hf["rw_means_linear"][()])
            rw_means_linear_err.append(hf["rw_means_linear_err"][()])
            rw_vars_linear.append(hf["rw_vars_linear"][()])
            rw_vars_linear_err.append(hf["rw_vars_linear_err"][()])
            pseudo_bcs_b1_bs.append(hf["pseudo_bc_bs"][()])
    else:
        temp = np.array(h[l])
        rw_means_linear_temp, rw_means_linear_temp_err, rw_vars_linear_temp, rw_vars_linear_temp_err, pseudo_bc_temp_bs = reweight.reweight(
            temp[:, :, 1], delta_E[l], betas_k[l], rw_betas_linear[l], N_bootstraps=N_bootstraps, tol=tol
        )
        with h5py.File(filename, "w") as hf:
            hf.create_dataset("rw_means_linear", data=rw_means_linear_temp)
            hf.create_dataset("rw_means_linear_err", data=rw_means_linear_temp_err)
            hf.create_dataset("rw_vars_linear", data=rw_vars_linear_temp)
            hf.create_dataset("rw_vars_linear_err", data=rw_vars_linear_temp_err)
            hf.create_dataset("pseudo_bc_bs", data=pseudo_bc_temp_bs)
        rw_means_linear.append(rw_means_linear_temp)
        rw_vars_linear.append(rw_vars_linear_temp)
        rw_means_linear_err.append(rw_means_linear_temp_err)
        rw_vars_linear_err.append(rw_vars_linear_temp_err)
        pseudo_bcs_b1_bs.append(pseudo_bc_temp_bs)
rw_means_linear_b1 = np.array(rw_means_linear)
rw_vars_linear_b1 = np.array(rw_vars_linear)
rw_means_linear_err_b1 = np.array(rw_means_linear_err)
rw_vars_linear_err_b1 = np.array(rw_vars_linear_err)
pseudo_bcs_b1_bs = np.array(pseudo_bcs_b1_bs)
rw_means_linear_b1.shape, rw_means_linear_err_b1.shape, rw_vars_linear_b1.shape, rw_vars_linear_err_b1.shape, pseudo_bcs_b1_bs.shape

In [ ]:
for l in range(len(Ls)):
    plt.scatter(rw_betas_linear[l], rw_means_linear_b1[l] / vol[l], s=1, color=colors[l], alpha=0.1)
    plt.errorbar(rw_betas_linear[l], rw_means_linear_b1[l] / vol[l], ls="None", yerr=rw_means_linear_err_b1[l] / vol[l], capsize=0, color=colors[l], alpha=0.05)
    x = betas_k[l]
    temp = np.array(h[l])
    y = np.mean(temp[:, :, 1], axis=1) / vol[l]
    plt.scatter(
        x,
        y,
        s=markersize,
        marker=fmt[l],
        label=r"$Nt, Ns = " + str(int(Ls[l][0])) + "," + str(int(Ls[l][1])) + "$",
        color=colors[l],
    )
    means = [
        np.mean(b1_bootstrap[i][l] , axis=1) / vol[l] for i in range(len(b1_bootstrap))
    ]
    yerr = np.std(means, axis=0)
    plt.errorbar(x, y, ls="None", yerr=yerr, capsize=2, color=colors[l], alpha=0.25)

plt.xlabel(r"$\beta$")
plt.ylabel(r"$\rho_{1}$")
plt.legend(loc="best")
plt.tight_layout()
plt.locator_params(axis="x", nbins=10)
ax = plt.gca()

# plt.savefig("../reports/figures/p_0_scatter_rw.pdf", format="pdf")
plt.show()

In [ ]:
for l in range(len(Ls)):
    plt.scatter(rw_betas_linear[l], (rw_vars_linear_b1[l]/vol[l]), s=1, color=colors[l], alpha=0.1)
    plt.errorbar(rw_betas_linear[l], (rw_vars_linear_b1[l]/vol[l]), ls="None", yerr=rw_means_linear_err_b1[l]/np.sqrt(vol[l]), capsize=0, color=colors[l], alpha=0.05)
    x = betas_k[l]
    temp = np.array(h[l])
    y = np.var(temp[:, :, 1], axis=1) / vol[l]
    plt.scatter(
        x,
        y,
        s=markersize,
        marker=fmt[l],
        label=r"$Nt, Ns = " + str(int(Ls[l][0])) + "," + str(int(Ls[l][1])) + "$",
        color=colors[l],
    )
    vars = [
        np.var(b1_bootstrap[i][l], axis=1) / vol[l]  for i in range(len(b1_bootstrap))
    ]
    yerr = np.std(vars, axis=0)
    plt.errorbar(x, y, ls="None", yerr=yerr, capsize=capsize, color=colors[l], alpha=0.25)

plt.xlabel(r"$\beta$")
plt.ylabel(r"$\chi_{1}$")
plt.legend(loc="best")
plt.tight_layout()
plt.locator_params(axis="x", nbins=10)
ax = plt.gca()
# plt.savefig("../reports/figures/X_1_scatter_rw.pdf", format="pdf")
plt.show()

In [ ]:
yerr_b1 = np.std(pseudo_bcs_b1_bs,axis=1)
pseudo_bcs_b1 = np.mean(pseudo_bcs_b1_bs,axis=1)
yerr_b1, pseudo_bcs_b1

In [ ]:
x = 1/vol
y = pseudo_bcs_b1
plt.scatter(x, y, s=10)
plt.errorbar(x, y, yerr=yerr_b1, fmt='.',capsize=3)
plt.axhspan(lit_beta_min, lit_beta_max, color="purple", alpha=0.15)
plt.axhline(literature_beta, color="purple", alpha=0.3)

Reweight $\lambda$

In [ ]:
# Reweighting beta values for each lattice size
## to stabilise numerics for smaller lattices sizes
betas_k = [
    np.array(
        [
            5.68, 5.6825, 5.685, 5.6875, 5.69, 5.6925, 5.695, 5.6975, 5.7, 5.705
        ]
    ),
    np.array(
        [
            5.675, 5.6800, 5.685, 5.6900, 5.691, 5.692, 5.693, 5.694, 5.695, 5.7000
        ]
    ),
    np.array(
        [
            5.691, 5.6913, 5.6915, 5.6917, 5.6919, 5.6921, 5.6923, 5.6925, 5.693, 5.6935
        ]
    ),
    np.array(
        [
            5.691, 5.6915, 5.6918, 5.692, 5.6921, 5.6922, 5.6923, 5.6925, 5.6928, 5.693
        ]
    ),
    np.array(
        [
            5.6915, 5.692, 5.6922, 5.6924, 5.6926, 5.6927, 5.6928, 5.693, 5.6932, 5.6935
        ]
    ),
]


In [ ]:
# Error bars are computed via the bootstrapping method

# Generate bootstraps
np.random.seed(bootstrap_seed)
bootstraps = [
    [
        [
            np.sort(np.random.choice(range(N), size=N, replace=True))
            for i in range(N_bootstraps)
        ]
        for b in range(len(betas_k[l]))
    ]
    for l in range(len(Ls))
]

In [ ]:
h = []
for l in range(len(Ls)):
    h_l = []
    for b in betas_k[l]:
        # Load Betti number HDF5 file if exists (else compute and save)
        filename = f"../data/observables/betti/{Ls[l][0]}.{Ls[l][1]}.{Ls[l][2]}.{Ls[l][3]}/betti_NtNsNsNs={Ls[l][0]}{Ls[l][1]}{Ls[l][2]}{Ls[l][3]}_b={b:.4f}.csv"
        # print(filename)
        if os.path.exists(filename):
            h_b = np.loadtxt(filename, delimiter=",", dtype=int)
        h_l.append(h_b)
    h.append(h_l)

In [ ]:
# Skip header line + start at sweep 10_000
E_l = []
for l in range(len(Ls)):
    print(Ls[l])
    E_b = []
    for b in range(len(betas_k[l])):
        # print(betas_k[l][b])
        averages = []
        with open(
            f"../data/observables/dati/{Ls[l][0]}.{Ls[l][1]}.{Ls[l][2]}.{Ls[l][3]}/dati.dat{betas_k[l][b]:.4f}",
            "r",
        ) as file:
            for line in file:
                elements = line.split()
                averages.append(
                    1.0 - (0.5 * (np.float64(elements[0]) + np.float64(elements[1])))
                )
        total_action = 6.0 * vol_tot[l] * np.array(averages, dtype=np.float64)
        E_b.append(total_action[dati_head : (dati_head + N)])
    E_l.append(E_b)
E = E_l

In [ ]:
# Compute delta_S for use in multiple histogram reweighting
E_bar = [np.mean(E[l], axis=0) for l in range(len(Ls))]
delta_E = [E[l] - E_bar[l][np.newaxis, :] for l in range(len(Ls))]

In [ ]:
# Generate b1 bootstrap distribution
ratio01_bootstrap = [
    [
        np.array([(h[l][b][bootstraps[l][b][i]][:, 0] / h[l][b][bootstraps[l][b][i]][:, 1]) for b in range(len(betas_k[l]))])
        for l in range(len(Ls))
    ]
    for i in range(N_bootstraps)
]

In [ ]:
# whole range of betas
rw_betas_linear = [
    np.linspace(betas_k[l][0], betas_k[l][-1], n_part, dtype=np.float64) for l in range(len(Ls))]

In [ ]:
rw_means_linear = []
rw_vars_linear = []
rw_means_linear_err = []
rw_vars_linear_err = []
pseudo_bcs_r01_bs = []
for l in range(len(Ls)):
    print(l)
    filename = f"../cached_data/cache_rw_{np.min(rw_betas_linear[l]):.4f}-{np.max(rw_betas_linear[l]):.4f}_NtNsNsNs={Ls[l][0]}{Ls[l][1]}{Ls[l][2]}{Ls[l][3]}_r01.h5"
    if os.path.exists(filename):
        with h5py.File(filename, "r") as hf:
            rw_means_linear.append(hf["rw_means_linear"][()])
            rw_means_linear_err.append(hf["rw_means_linear_err"][()])
            rw_vars_linear.append(hf["rw_vars_linear"][()])
            rw_vars_linear_err.append(hf["rw_vars_linear_err"][()])
            pseudo_bcs_r01_bs.append(hf["pseudo_bc_bs"][()])
    else:
        temp = np.array(h[l])
        rw_means_linear_temp, rw_means_linear_temp_err, rw_vars_linear_temp, rw_vars_linear_temp_err, pseudo_bc_temp_bs = reweight.reweight(
            (temp[:, :, 0] / temp[:, :, 1]), delta_E[l], betas_k[l], rw_betas_linear[l], N_bootstraps=N_bootstraps, tol=tol
        )
        with h5py.File(filename, "w") as hf:
            hf.create_dataset("rw_means_linear", data=rw_means_linear_temp)
            hf.create_dataset("rw_means_linear_err", data=rw_means_linear_temp_err)
            hf.create_dataset("rw_vars_linear", data=rw_vars_linear_temp)
            hf.create_dataset("rw_vars_linear_err", data=rw_vars_linear_temp_err)
            hf.create_dataset("pseudo_bc_bs", data=pseudo_bc_temp_bs)
        rw_means_linear.append(rw_means_linear_temp)
        rw_vars_linear.append(rw_vars_linear_temp)
        rw_means_linear_err.append(rw_means_linear_temp_err)
        rw_vars_linear_err.append(rw_vars_linear_temp_err)
        pseudo_bcs_r01_bs.append(pseudo_bc_temp_bs)
rw_means_linear_r01 = np.array(rw_means_linear)
rw_vars_linear_r01 = np.array(rw_vars_linear)
rw_means_linear_err_r01 = np.array(rw_means_linear_err)
rw_vars_linear_err_r01 = np.array(rw_vars_linear_err)
pseudo_bcs_r01_bs = np.array(pseudo_bcs_r01_bs)
rw_means_linear_r01.shape, rw_means_linear_err_r01.shape, rw_vars_linear_r01.shape, rw_vars_linear_err_r01.shape, pseudo_bcs_r01_bs.shape

In [ ]:
for l in range(len(Ls)):
    plt.scatter(rw_betas_linear[l], rw_means_linear_r01[l], s=1, color=colors[l], alpha=0.1)
    plt.errorbar(rw_betas_linear[l], rw_means_linear_r01[l], ls="None", yerr=rw_means_linear_err_r01[l], capsize=0, color=colors[l], alpha=0.05)
    x = betas_k[l]
    temp = np.array(h[l])
    y = np.mean(temp[:, :, 0] / temp[:, :, 1], axis=1)
    plt.scatter(
        x,
        y,
        s=markersize,
        marker=fmt[l],
        label=r"$Nt, Ns = " + str(int(Ls[l][0])) + "," + str(int(Ls[l][1])) + "$",
        color=colors[l],
    )
    means = [
        np.mean(ratio01_bootstrap[i][l] , axis=1) for i in range(len(ratio01_bootstrap))
    ]
    yerr = np.std(means, axis=0)
    plt.errorbar(x, y, ls="None", yerr=yerr, capsize=2, color=colors[l], alpha=0.25)

plt.xlabel(r"$\beta$")
plt.ylabel(r"$\lambda $")
plt.legend(loc="best")
plt.tight_layout()
plt.locator_params(axis="x", nbins=10)
ax = plt.gca()

# plt.savefig("../reports/figures/r01_scatter_rw.pdf", format="pdf")
plt.show()

In [ ]:
for l in range(len(Ls)):
    plt.scatter(rw_betas_linear[l], (rw_vars_linear_r01[l] * vol[l]), s=1, color=colors[l], alpha=0.1)
    plt.errorbar(rw_betas_linear[l], (rw_vars_linear_r01[l] * vol[l]), ls="None", yerr=(rw_means_linear_err_r01[l] * np.sqrt(vol[l])), capsize=0, color=colors[l], alpha=0.05)
    x = betas_k[l]
    temp = np.array(h[l])
    y = np.var(temp[:, :, 0] / temp[:, :, 1], axis=1) * vol[l]
    plt.scatter(
        x,
        y,
        s=markersize,
        marker=fmt[l],
        label=r"$Nt, Ns = " + str(int(Ls[l][0])) + "," + str(int(Ls[l][1])) + "$",
        color=colors[l],
    )
    vars = [
        np.var(ratio01_bootstrap[i][l], axis=1) * vol[l]  for i in range(len(ratio01_bootstrap))
    ]
    yerr = np.std(vars, axis=0)
    plt.errorbar(x, y, ls="None", yerr=yerr, capsize=capsize, color=colors[l], alpha=0.25)

plt.xlabel(r"$\beta$")
plt.ylabel(r"$\chi_{01}$")
plt.legend(loc="best")
plt.tight_layout()
plt.locator_params(axis="x", nbins=10)
ax = plt.gca()
# plt.savefig("../reports/figures/X_01_scatter_rw.pdf", format="pdf")
plt.show()

In [ ]:
yerr_r01 = np.std(pseudo_bcs_r01_bs,axis=1)
pseudo_bcs_r01 = np.mean(pseudo_bcs_r01_bs,axis=1)
yerr_r01, pseudo_bcs_r01

In [ ]:
x = 1/vol
y = pseudo_bcs_r01
plt.scatter(x, y, s=10)
plt.errorbar(x, y, yerr=yerr_r01, fmt='.',capsize=3)
plt.axhspan(lit_beta_min, lit_beta_max, color="purple", alpha=0.15)
plt.axhline(literature_beta, color="purple", alpha=0.3)

# Finite-size scaling analysis

In [ ]:
import polars as pl
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
from scipy.optimize import brentq

In [ ]:
def format_NIST(m, u):
    """Formats measurement and standard error in NIST:
    https://physics.nist.gov/cgi-bin/cuu/Info/Constants/definitions.html
    """
    m = float(m)
    u = float(u)
    u_order = int(np.floor(np.log10(u)))
    round_m = "{:.{}f}".format(m, -1 * u_order)
    round_u = str(int(np.round(u * 10 ** (-u_order))))
    return round_m + "(" + round_u + ")"


In [ ]:
def fss(type, Ls_for_reg, deg, y, y_bs):
    """Function to output range, k_max, reduced_chi2, AIC, beta_c, beta_c_err.
       Returns None if an unacceptable fit (with internal extremum) is detected.
    """
    # Perform polynomial regression
    poly_features = PolynomialFeatures(degree=deg, include_bias=False)
    X_poly = poly_features.fit_transform(x.reshape(-1, 1))
    lin_reg = LinearRegression().fit(X_poly[Ls_for_reg], y[Ls_for_reg])

    if deg >= 2:
        # Descending order: [a_d, ..., a_1, a_0]
        p_coefs_desc = list(lin_reg.coef_[::-1]) + [lin_reg.intercept_]
        der_coefs = np.polyder(p_coefs_desc)
        roots = np.roots(der_coefs)
        real_roots = roots[np.isreal(roots)].real
        x_min = 0.0
        x_max = x.max()
        # Check if any real root is strictly inside the data range
        if np.any((real_roots > x_min) & (real_roots < x_max)):
            print("Rejecting fit; found internal extremum at", real_roots)
            return None

    # Compute intercept error via bootstrap
    intercepts = [
        LinearRegression().fit(X_poly[Ls_for_reg], y_bs[i][Ls_for_reg]).intercept_
        for i in range(N_bootstraps)
    ]
    intercept_err = np.sqrt(np.var(intercepts))

    # Compute reduced chi-squared
    f_obs = y[Ls_for_reg]
    f_exp = lin_reg.predict(X_poly[Ls_for_reg])
    var = np.var(y_bs[:, Ls_for_reg], axis=0)
    chi2 = np.sum((f_obs - f_exp) ** 2 / var)

    # Calculate AIC
    n_par = deg + 1
    n_data = len(Ls_for_reg)
    aic_unnorm = np.exp(-0.5 * (chi2 + 2 * n_par - n_data))

    return (
        type,
        str(Ls[Ls_for_reg][0]),
        str(Ls[Ls_for_reg][-1]),
        deg,
        chi2,
        aic_unnorm,
        lin_reg.intercept_,
        list(lin_reg.coef_),
        intercept_err,
        f"{chi2:.3f}",
        f"{aic_unnorm:.3f}",
        format_NIST(lin_reg.intercept_, intercept_err),
    )


In [ ]:
# Define the mixture CDF: sum_i w_i * N(x|mu_i, sigma_i)
def mixture_cdf(x, weights, means, sigmas):
    return np.sum(weights * norm.cdf(x, loc=means, scale=sigmas))

# Function to find x such that the CDF equals a target value.
def find_quantile(target, weights, means, sigmas, x_min, x_max):
    func = lambda x: mixture_cdf(x, weights, means, sigmas) - target
    return brentq(func, x_min, x_max)

In [ ]:
def format_asymmetric_error(lower, central, upper):
    """
    Given lower (X_16), central (X_50), and upper (X_84) values as floats,
    compute asymmetric uncertainties, round them to appropriate significant figures,
    and return a LaTeX-formatted string.
    """
    # Compute the deltas
    delta_minus = central - lower
    delta_plus = upper - central

    def round_uncertainty(d):
        """Round uncertainty to 1 significant figure, or 2 if first digit is 1."""
        if d == 0:
            return 0.0, 0
        exp = np.floor(np.log10(d))
        first_digit = d / (10 ** exp)
        sig_figs = 2 if 1 <= first_digit < 2 else 1
        # Number of decimal places for round()
        decimals = max(0, int(-exp + (sig_figs - 1)))
        return round(d, decimals), decimals

    rm, dec_m = round_uncertainty(delta_minus)
    rp, dec_p = round_uncertainty(delta_plus)
    # Decimal places for central value
    dec_c = max(dec_m, dec_p)
    central_rounded = round(central, dec_c)

    # Build the LaTeX string
    fmt = f"${central_rounded:.{dec_c}f}" \
          f"^{{+{rp:.{dec_c}f}}}_{{-{rm:.{dec_c}f}}}$"
          
    return fmt

$\rho_0$

In [ ]:
# Define the range of Ls values to be used in FSS
range_ = [[2, 3, 4], [1, 2, 3, 4], [0, 1, 2, 3, 4]]

# When computing results, filter out any models where fss returns None
results = []
for type, ps_bc, ps_bc_var in [("rho_b0", pseudo_bcs_b0, pseudo_bcs_b0_bs.T)]:
    for i in range(len(range_)):
        for d in range(1, len(range_[i]) - 1):
            model = fss(type, range_[i], d, ps_bc, ps_bc_var)
            if model is not None:
                results.append(model)

In [ ]:
df = pl.DataFrame(results)
df.columns = [
    "observable",
    "range_min",
    "range_max",
    "k_max",
    "chi2",
    "aic_unnorm",
    "beta_c",
    "coefficients",
    "beta_c_err",
    "str_chi2",
    "str_aic",
    "beta_c_NIST",
]

In [ ]:
df

In [ ]:
# Extract AIC, intercepts, and errors
weight_unnorms_b0 = np.array([model[5] for model in results])  # Assuming AIC is the 6th element
beta_c_values_b0 = np.array([model[6] for model in results])  # Intercept
beta_c_errs_b0 = np.array([model[8] for model in results])  # Intercept error

# Compute model weights (pr(M|D))
weights_b0 = weight_unnorms_b0 / np.sum(weight_unnorms_b0)
weights_b0

In [ ]:
# Define a search interval that should comfortably include the distribution.
x_min_search = np.min(beta_c_values_b0) - 5 * np.max(beta_c_errs_b0)
x_max_search = np.max(beta_c_values_b0) + 5 * np.max(beta_c_errs_b0)

# Compute the quantiles for the median and the 68% interval.
lower_val = find_quantile(0.16, weights_b0, beta_c_values_b0, beta_c_errs_b0, x_min_search, x_max_search)
median_val = find_quantile(0.5, weights_b0, beta_c_values_b0, beta_c_errs_b0, x_min_search, x_max_search)
upper_val = find_quantile(0.84, weights_b0, beta_c_values_b0, beta_c_errs_b0, x_min_search, x_max_search)

# Compute asymmetric errors (distance from the median to the lower and upper bounds)
error_lower = median_val - lower_val
error_upper = upper_val - median_val

In [ ]:
error_lower, median_val, error_upper

In [ ]:
latex_str = format_asymmetric_error(lower_val, median_val, upper_val)
print(latex_str)

In [ ]:
plt.figure(figsize=(8, 6))

plt.axhline(literature_beta, color="purple", linestyle=(0, (3, 1, 1, 1)), alpha=0.5, label=r"Liter. $\beta_c (\infty, N_t =$"+f"{Ls[0,0]}"+r"$)$ estimate")
plt.axhspan(lit_beta_min, lit_beta_max, color="purple", alpha=0.1, label=r"Liter. $\beta_c (\infty, N_t =$"+f"{Ls[0,0]}"+r"$)$ std. error")


# Plot each polynomial fit
for j,row in enumerate(df.rows(named=True)):
    intercept = row["beta_c"]
    coefficients = row["coefficients"]
    deg = row["k_max"]
    
    # Generate smooth x values for plotting the curve
    x_plot = np.linspace(0, max(x)*1.1, 300)
    
    # Manually compute polynomial: intercept + c1*x + c2*x² + ...
    y_plot = intercept + sum(
        coef * (x_plot ** (i + 1)) 
        for i, coef in enumerate(coefficients)
    )
    
    rangemin = list(map(int, row['range_min'].strip("[]").split()))[1]
    rangemax = list(map(int, row['range_max'].strip("[]").split()))[1]
    Ns_range = np.arange(rangemin, rangemax+1, Ls[0,0])
    Ns_range_str = ", ".join(str(l) for l in Ns_range)
    
    # Label the fit
    label = f"Polyn. degree: {deg}\n$N_s$ values: {Ns_range_str}"
    plt.plot(x_plot, y_plot, "--", label=label, alpha=0.7)
    
    
x = 1/vol
y = pseudo_bcs_b0
plt.errorbar(x, y, yerr=yerr_b0, fmt="^", label=r"$\beta_c ( N_s, N_t =$"+f"{Ls[0,0]}"+r"$)$", capsize=7, color="blue", markersize=7)
plt.errorbar(0, median_val, yerr=[[error_lower], [error_upper]], fmt=".", label=r"$\beta_c (\infty, N_t =$"+f"{Ls[0,0]}"+r"$)$ estimate", capsize=7, color="red", markersize=7)

plt.xlabel(r"$N_{s}^{-3}$",fontsize=30)
plt.ylabel(r"$\beta$", fontsize=30,rotation=0,labelpad=15)
plt.ylim([5.6920,5.6934])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

ax = plt.gca()
ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
ax.xaxis.get_offset_text().set_fontsize(20)
handles, labels = ax.get_legend_handles_labels()

is_fit = [lab.startswith("Polyn.") for lab in labels]
fit_handles  = [h for h,f in zip(handles,is_fit) if f]
fit_labels   = [l for l,f in zip(labels,is_fit) if f]
other_handles = [h for h,f in zip(handles,is_fit) if not f]
other_labels  = [l for l,f in zip(labels,is_fit) if not f]

leg1 = ax.legend(
    other_handles, other_labels,
    loc='upper left',
    frameon=True,
    ncol=1,
    fontsize=12
)
leg2 = ax.legend(
    fit_handles, fit_labels,
    loc='upper left',
    bbox_to_anchor=(0.0,0.75),
    frameon=True,
    title="Fits",
    ncol=1,
    fontsize=11
)
ax.add_artist(leg1)

plt.tight_layout()
plt.savefig(f"../reports/Nt={Ls[0,0]}/figures/fits_rho0.pdf", format="pdf",bbox_inches='tight',pad_inches=0.025)
plt.show()

$\rho_1$

In [ ]:
# Define the range of Ls values to be used in FSS
range_ = [[2, 3, 4], [1, 2, 3, 4], [0, 1, 2, 3, 4]]

# When computing results, filter out any models where fss returns None
results = []
for type, ps_bc, ps_bc_var in [("rho_b1", pseudo_bcs_b1, pseudo_bcs_b1_bs.T)]:
    for i in range(len(range_)):
        for d in range(1, len(range_[i]) - 1):
            model = fss(type, range_[i], d, ps_bc, ps_bc_var)
            if model is not None:
                results.append(model)

In [ ]:
df = pl.DataFrame(results)
df.columns = [
    "observable",
    "range_min",
    "range_max",
    "k_max",
    "chi2",
    "aic_unnorm",
    "beta_c",
    "coefficients",
    "beta_c_err",
    "str_chi2",
    "str_aic",
    "beta_c_NIST",
]

In [ ]:
df

In [ ]:
# Extract AIC, intercepts, and errors
weight_unnorms_b1 = np.array([model[5] for model in results])  # Assuming AIC is the 6th element
beta_c_values_b1 = np.array([model[6] for model in results])  # Intercept
beta_c_errs_b1 = np.array([model[8] for model in results])  # Intercept error

# Compute model weights (pr(M|D))
weights_b1 = weight_unnorms_b1 / np.sum(weight_unnorms_b1)
weights_b1

In [ ]:
# Define a search interval that should comfortably include the distribution.
x_min_search = np.min(beta_c_values_b1) - 5 * np.max(beta_c_errs_b1)
x_max_search = np.max(beta_c_values_b1) + 5 * np.max(beta_c_errs_b1)

# Compute the quantiles for the median and the 68% interval.
lower_val = find_quantile(0.16, weights_b1, beta_c_values_b1, beta_c_errs_b1, x_min_search, x_max_search)
median_val = find_quantile(0.5, weights_b1, beta_c_values_b1, beta_c_errs_b1, x_min_search, x_max_search)
upper_val = find_quantile(0.84, weights_b1, beta_c_values_b1, beta_c_errs_b1, x_min_search, x_max_search)

# Compute asymmetric errors (distance from the median to the lower and upper bounds)
error_lower = median_val - lower_val
error_upper = upper_val - median_val

In [ ]:
error_lower, median_val, error_upper

In [ ]:
latex_str = format_asymmetric_error(lower_val, median_val, upper_val)
print(latex_str)

In [ ]:
plt.figure(figsize=(8, 6))

plt.axhline(literature_beta, color="purple", linestyle=(0, (3, 1, 1, 1)), alpha=0.5, label=r"Liter. $\beta_c (\infty, N_t =$"+f"{Ls[0,0]}"+r"$)$ estimate")
plt.axhspan(lit_beta_min, lit_beta_max, color="purple", alpha=0.1, label=r"Liter. $\beta_c (\infty, N_t =$"+f"{Ls[0,0]}"+r"$)$ std. error")


# Plot each polynomial fit
for j,row in enumerate(df.rows(named=True)):
    intercept = row["beta_c"]
    coefficients = row["coefficients"]
    deg = row["k_max"]
    
    # Generate smooth x values for plotting the curve
    x_plot = np.linspace(0, max(x)*1.1, 300)
    
    # Manually compute polynomial: intercept + c1*x + c2*x² + ...
    y_plot = intercept + sum(
        coef * (x_plot ** (i + 1)) 
        for i, coef in enumerate(coefficients)
    )
    
    rangemin = list(map(int, row['range_min'].strip("[]").split()))[1]
    rangemax = list(map(int, row['range_max'].strip("[]").split()))[1]
    Ns_range = np.arange(rangemin, rangemax+1, Ls[0,0])
    Ns_range_str = ", ".join(str(l) for l in Ns_range)
    
    # Label the fit
    label = f"Polyn. degree: {deg}\n$N_s$ values: {Ns_range_str}"
    plt.plot(x_plot, y_plot, "--", label=label, alpha=0.7)

x = 1/vol
y = pseudo_bcs_b1
plt.errorbar(x, y, yerr=yerr_b1, fmt="^", label=r"$\beta_c ( N_s, N_t =$"+f"{Ls[0,0]}"+r"$)$", capsize=7, color="blue", markersize=7)
plt.errorbar(0, median_val, yerr=[[error_lower], [error_upper]], fmt=".", label=r"$\beta_c (\infty, N_t =$"+f"{Ls[0,0]}"+r"$)$ estimate", capsize=7, color="red", markersize=7)

plt.xlabel(r"$N_{s}^{-3}$",fontsize=30)
plt.ylabel(r"$\beta$",fontsize=30,rotation=0,labelpad=15)
plt.ylim([5.6900,5.6928])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

ax = plt.gca()
ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
ax.xaxis.get_offset_text().set_fontsize(20)
handles, labels = ax.get_legend_handles_labels()

is_fit = [lab.startswith("Polyn.") for lab in labels]
fit_handles  = [h for h,f in zip(handles,is_fit) if f]
fit_labels   = [l for l,f in zip(labels,is_fit) if f]
other_handles = [h for h,f in zip(handles,is_fit) if not f]
other_labels  = [l for l,f in zip(labels,is_fit) if not f]

leg1 = ax.legend(
    other_handles, other_labels,
    loc='lower left',
    frameon=True,
    ncol=2,
    fontsize=13
)
leg2 = ax.legend(
    fit_handles, fit_labels,
    loc='lower left',
    bbox_to_anchor=(0.0, 0.15),
    frameon=True,
    title="Fits",
    ncol=1,
    fontsize=10.5
)
ax.add_artist(leg1)

plt.tight_layout()
plt.savefig(f"../reports/Nt={Ls[0,0]}/figures/fits_rho1.pdf",format="pdf",bbox_inches='tight',pad_inches=0.025)
plt.show()

$\lambda$

In [ ]:
# Define the range of Ls values to be used in FSS
range_ = [[2, 3, 4], [1, 2, 3, 4], [0, 1, 2, 3, 4]]

# When computing results, filter out any models where fss returns None
results = []
for type, ps_bc, ps_bc_var in [("r01", pseudo_bcs_r01, pseudo_bcs_r01_bs.T)]:
    for i in range(len(range_)):
        for d in range(1, len(range_[i]) - 1):
            model = fss(type, range_[i], d, ps_bc, ps_bc_var)
            if model is not None:
                results.append(model)

In [ ]:
df = pl.DataFrame(results)
df.columns = [
    "observable",
    "range_min",
    "range_max",
    "k_max",
    "chi2",
    "aic_unnorm",
    "beta_c",
    "coefficients",
    "beta_c_err",
    "str_chi2",
    "str_aic",
    "beta_c_NIST",
]

In [ ]:
df

In [ ]:
# Extract AIC, intercepts, and errors
weight_unnorms_b1 = np.array([model[5] for model in results])  # Assuming AIC is the 6th element
beta_c_values_b1 = np.array([model[6] for model in results])  # Intercept
beta_c_errs_b1 = np.array([model[8] for model in results])  # Intercept error

# Compute model weights (pr(M|D))
weights_b1 = weight_unnorms_b1 / np.sum(weight_unnorms_b1)
weights_b1

In [ ]:
# Define a search interval that should comfortably include the distribution.
x_min_search = np.min(beta_c_values_b1) - 5 * np.max(beta_c_errs_b1)
x_max_search = np.max(beta_c_values_b1) + 5 * np.max(beta_c_errs_b1)

# Compute the quantiles for the median and the 68% interval.
lower_val = find_quantile(0.16, weights_b1, beta_c_values_b1, beta_c_errs_b1, x_min_search, x_max_search)
median_val = find_quantile(0.5, weights_b1, beta_c_values_b1, beta_c_errs_b1, x_min_search, x_max_search)
upper_val = find_quantile(0.84, weights_b1, beta_c_values_b1, beta_c_errs_b1, x_min_search, x_max_search)

# Compute asymmetric errors (distance from the median to the lower and upper bounds)
error_lower = median_val - lower_val
error_upper = upper_val - median_val

In [ ]:
error_lower, median_val, error_upper

In [ ]:
latex_str = format_asymmetric_error(lower_val, median_val, upper_val)
print(latex_str)

In [ ]:
plt.figure(figsize=(8, 6))

plt.axhline(literature_beta, color="purple", linestyle=(0, (3, 1, 1, 1)), alpha=0.5, label=r"Liter. $\beta_c (\infty, N_t =$"+f"{Ls[0,0]}"+r"$)$ estimate")
plt.axhspan(lit_beta_min, lit_beta_max, color="purple", alpha=0.1, label=r"Liter. $\beta_c (\infty, N_t =$"+f"{Ls[0,0]}"+r"$)$ std. error")

# Plot each polynomial fit
for j,row in enumerate(df.rows(named=True)):
    intercept = row["beta_c"]
    coefficients = row["coefficients"]
    deg = row["k_max"]
    
    # Generate smooth x values for plotting the curve
    x_plot = np.linspace(0, max(x)*1.1, 300)
    
    # Manually compute polynomial: intercept + c1*x + c2*x² + ...
    y_plot = intercept + sum(
        coef * (x_plot ** (i + 1)) 
        for i, coef in enumerate(coefficients)
    )
    
    rangemin = list(map(int, row['range_min'].strip("[]").split()))[1]
    rangemax = list(map(int, row['range_max'].strip("[]").split()))[1]
    Ns_range = np.arange(rangemin, rangemax+1, Ls[0,0])
    Ns_range_str = ", ".join(str(l) for l in Ns_range)
    
    # Label the fit
    label = f"Polyn. degree: {deg}\n$N_s$ values: {Ns_range_str}"
    plt.plot(x_plot, y_plot, "--", label=label, alpha=0.7)
    
    
x = 1/vol
y = pseudo_bcs_r01
plt.errorbar(x, y, yerr=yerr_r01, fmt="^", label=r"$\beta_c (N_s , N_t =$"+f"{Ls[0,0]}"+r"$)$", capsize=7, color="blue", markersize=7)
plt.errorbar(0, median_val, yerr=[[error_lower], [error_upper]], fmt=".", label=r"$\beta_c (\infty, N_t =$"+f"{Ls[0,0]}"+r"$)$ estimate", capsize=7, color="red", markersize=7)

plt.xlabel(r"$N_{s}^{-3}$",fontsize=30)
plt.ylabel(r"$\beta$", fontsize=30,rotation=0,labelpad=15)
plt.ylim([5.6918, 5.6950])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)


ax = plt.gca()
ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
ax.xaxis.get_offset_text().set_fontsize(20)
handles, labels = ax.get_legend_handles_labels()

is_fit = [lab.startswith("Polyn.") for lab in labels]
fit_handles  = [h for h,f in zip(handles,is_fit) if f]
fit_labels   = [l for l,f in zip(labels,is_fit) if f]
other_handles = [h for h,f in zip(handles,is_fit) if not f]
other_labels  = [l for l,f in zip(labels,is_fit) if not f]

leg1 = ax.legend(
    other_handles, other_labels,
    loc='upper left',
    frameon=True,
    ncol=1,
    fontsize=12
)
leg2 = ax.legend(
    fit_handles, fit_labels,
    loc='upper left',
    bbox_to_anchor=(0.0, 0.75),
    frameon=True,
    title="Fits",
    ncol=1,
    fontsize=11.5
)
ax.add_artist(leg1)

plt.tight_layout()
plt.savefig(f"../reports/Nt={Ls[0,0]}/figures/fits_r01.pdf",format="pdf",bbox_inches='tight',pad_inches=0.025)
plt.show()